In [1]:
# cell 1

import pandas as pd
import numpy as np
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# 1. Load count matrix using TPM
data_path = '/Users/esrataner/Documents/DATA1030/count_matrix_120'
file_list = [f for f in os.listdir(data_path) if f.endswith('.tsv')]

def read_tpm_file(filename):
    df = pd.read_csv(os.path.join(data_path, filename), sep='\t',
                     usecols=['gene_id', 'TPM'])
    df = df.set_index('gene_id')
    sample_id = filename.replace('.tsv', '')
    df.columns = [sample_id]
    return df

dfs = [read_tpm_file(f) for f in file_list]
merged = pd.concat(dfs, axis=1)  # genes × samples
print("Raw merged shape:", merged.shape)

# 2. remove all spike-in genes 
# these are technical control genes - not needed for ML 
spikein_mask = merged.index.str.lower().str.contains(
    "spikein|ercc"
)

print("Spike-in genes detected:", spikein_mask.sum())

merged = merged.loc[~spikein_mask]
print("After removing spike-ins:", merged.shape)

# 3. Remove all-zero genes
zero_across_all = (merged == 0).all(axis=1)
print("Genes zero everywhere:", zero_across_all.sum())

expr = merged[~zero_across_all]
print("After removing all-zero genes:", expr.shape)

# 4. log1p(TPM) transform — biological normalization
expr_log = np.log1p(expr)

# NOTE: try graphing log1p and log 
# depends on if data is close to zero or not 

Raw merged shape: (59526, 120)
Spike-in genes detected: 97
After removing spike-ins: (59429, 120)
Genes zero everywhere: 14289
After removing all-zero genes: (45140, 120)


In [2]:
# cell 2

meta = pd.read_csv(
    '/Users/esrataner/Documents/DATA1030/tsv/experiment_report_2025_120.tsv',
    sep='\t', skiprows=1
)

# extract sex
meta['Sex'] = meta['Biosample summary'].str.extract(r'(?i)\b(female|male)\b')
meta['Sex'] = meta['Sex'].str.lower()

# parse numeric age
def parse_age(a):
    if pd.isna(a):
        return None
    nums = re.findall(r'\d+', str(a))
    return int(nums[0]) if nums else None

meta['Age'] = meta['Biosample age'].apply(parse_age)

# Age bins
def age_bin(x):
    if pd.isna(x): return None
    if x < 65: return "60-64"
    if x < 70: return "65-69"
    if x < 75: return "70-74"
    if x < 80: return "75-79"
    if x < 85: return "80-84"
    if x < 90: return "85-89"
    return "90+"

meta['Age_Ordinal'] = meta['Age'].apply(age_bin)

age_order = ["60-64","65-69","70-74","75-79","80-84","85-89","90+"]
meta['Age_Ordinal'] = pd.Categorical(
    meta['Age_Ordinal'], categories=age_order, ordered=True
)

# FIX!!  Extract *all* ENCFF file IDs (not just first)
# Find ALL ENCFF IDs
meta['File_IDs'] = meta['Files'].str.findall(r'ENCFF\w+')

# Expand metadata so each row contains ONE ENCFF ID
meta_expanded = meta.explode('File_IDs')

# Keep only relevant fields
meta_expanded = meta_expanded[
    ['Accession', 'Sex', 'Age_Ordinal', 'Biosample accession', 'File_IDs']
]

# Drop rows missing file IDs
meta_expanded = meta_expanded.dropna(subset=['File_IDs'])

print("Expanded metadata shape:", meta_expanded.shape)
# Expanded metadata shape: (1220, 5)

# exp 
# in the raw ENCODE metadata, each biological sample (ENCSR) has multiple ENCFF files 
# (technical sequencing files: BAMs, fastqs, quantifications, etc)

# After expanding, u get 1 row per ENCFF file
# so if each ENCSR has ~10 ENCFF files ->  120 biological samples become ~1220 rows.
# so 1220 is NOT 120 samples - it is technical files, NOT samples

# MOST IMPORTANTLY - do NOT need GroupShuffleSplit
# 1. each ENCSR is represented only once
# 2. no ENCFF technical replicates remain#
# 3. df_full has 1 row per biological sample





Expanded metadata shape: (1220, 5)


In [3]:
# cell 3 

# List of ENCFF IDs found in expression matrix
expr_samples = expr_log.columns.tolist()

# Match metadata rows where File_IDs map to expression samples
meta_matched = meta_expanded[meta_expanded['File_IDs'].isin(expr_samples)].copy()
print("Matched metadata rows:", meta_matched.shape)

# Step 1: Relabel expression matrix columns (ENCFF → ENCSR)
rename_dict = dict(zip(meta_matched['File_IDs'], meta_matched['Accession']))
expr_labeled = expr_log.rename(columns=rename_dict)

# Step 2: Identify replicate sample IDs (appear twice)
replicate_accessions = expr_labeled.columns[
    expr_labeled.columns.duplicated()
].unique()

print("Replicate sample accessions:", replicate_accessions.tolist())

# Step 3: DROP the replicate sample completely
expr_no_reps = expr_labeled.drop(columns=list(replicate_accessions))
meta_no_reps = meta_matched[
    ~meta_matched['Accession'].isin(replicate_accessions)
].copy()

print("Expression shape after removing replicates:", expr_no_reps.shape)
print("Metadata shape after removing replicates:", meta_no_reps.shape)

# Step 4: Align expression with metadata
expr_T = expr_no_reps.T
expr_T.index.name = 'Accession'
expr_T = expr_T.reset_index()

# Deduplicate final metadata
meta_unique = meta_no_reps.drop_duplicates(subset=['Accession'])

# Step 5: Merge into final ML-ready dataset
df_full = expr_T.merge(
    meta_unique.drop(columns=['File_IDs']),
    on='Accession',
    how='inner'
).set_index('Accession')

print("Final merged dataset shape:", df_full.shape)
print(df_full.head())
# cell 4 

# TRAIN / VAL / TEST SPLIT
from sklearn.model_selection import train_test_split

# (1) Define features + target
X = df_full.drop(columns=["Sex", "Biosample accession"]) #string carried over 
y = df_full["Sex"].map({"female": 0, "male": 1}).astype(int)

y = y.astype(int)

# (2) First split: Train vs Temp (Val+Test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# (3) Split Temp into Validation + Test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

# Print shapes
print("Dataset Splits:")
print("Train:", X_train.shape, "Target:", y_train.shape)
print("Val:  ", X_val.shape,   "Target:", y_val.shape)
print("Test: ", X_test.shape,  "Target:", y_test.shape)


Matched metadata rows: (120, 5)
Replicate sample accessions: []
Expression shape after removing replicates: (45140, 120)
Metadata shape after removing replicates: (120, 5)
Final merged dataset shape: (120, 45143)
             13023     26893  30031  30958     30964  ENSG00000000003.14  \
Accession                                                                  
ENCSR800PJQ    0.0  0.506818    0.0    0.0  0.000000            1.348073   
ENCSR133PLR    0.0  0.000000    0.0    0.0  2.324347            1.340250   
ENCSR418WMG    0.0  0.000000    0.0    0.0  0.000000            1.181727   
ENCSR013HWB    0.0  0.438255    0.0    0.0  0.000000            1.313724   
ENCSR693KOP    0.0  0.000000    0.0    0.0  0.000000            1.081805   

             ENSG00000000005.5  ENSG00000000419.12  ENSG00000000457.13  \
Accession                                                                
ENCSR800PJQ           0.000000            2.157559            1.444563   
ENCSR133PLR           0.122218  

In [4]:
# cell 4 

# TRAIN / VAL / TEST SPLIT
from sklearn.model_selection import train_test_split

# (1) Define features + target
X = df_full.drop(columns=["Sex", "Biosample accession"]) #string carried over 
y = df_full["Sex"].map({"female": 0, "male": 1}).astype(int)

y = y.astype(int)

# (2) First split: Train vs Temp (Val+Test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# (3) Split Temp into Validation + Test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

# Print shapes
print("Dataset Splits:")
print("Train:", X_train.shape, "Target:", y_train.shape)
print("Val:  ", X_val.shape,   "Target:", y_val.shape)
print("Test: ", X_test.shape,  "Target:", y_test.shape)


Dataset Splits:
Train: (84, 45141) Target: (84,)
Val:   (18, 45141) Target: (18,)
Test:  (18, 45141) Target: (18,)


In [5]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import average_precision_score

dummy = DummyClassifier(strategy="stratified", random_state=42)
dummy.fit(X_train, y_train)

y_dummy_prob = dummy.predict_proba(X_test)[:, 1]
baseline_pr_auc = average_precision_score(y_test, y_dummy_prob)

print("Baseline PR-AUC:", baseline_pr_auc)


Baseline PR-AUC: 0.22916666666666666
